# Grupo Bimbo Inventory Demand Prediction Challenge

Data Exploration

## Loading the datasets

In [6]:
import pandas as pd
import numpy as np

In [7]:
train_test_dtypes = {
    "Semana": np.int8,
    "Agencia_ID": np.int16,
    "Canal_ID": np.int8,
    "Ruta_SAK": np.int16, 
    "Cliente_ID": np.int32,
    "Producto_ID": np.int32,
    "Venta_uni_hoy": np.int16, 
    "Venta_hoy": np.float32,
    "Dev_uni_proxima": np.int32,
    "Dev_proxima": np.float32,
    "Demanda_uni_equil": np.int16
}

In [8]:
train_data = pd.read_csv("Stage_0-Datasets/train.csv", dtype=train_test_dtypes)
test_data = pd.read_csv("Stage_0-Datasets/test.csv", dtype=train_test_dtypes)

In [9]:
dtypes_town_state = {"Agencia_ID": np.int16}
dtypes_client = {"Cliente_ID": np.int32}
dtypes_product = {"Producto_ID": np.int32}

In [10]:
town_state_data = pd.read_csv("Stage_0-Datasets/town_state.csv", dtype=dtypes_town_state)
client_data = pd.read_csv("Stage_0-Datasets/cliente_tabla.csv", dtype=dtypes_client)
product_data = pd.read_csv("Stage_0-Datasets/producto_tabla.csv", dtype=dtypes_product)

In [11]:
columns = {
    "Semana": "wk_no", 
    "Agencia_ID": "Sales Depot ID", 
    "Canal_ID": "Sales Channel ID", 
    "Ruta_SAK": "Route ID", 
    "Cliente_ID": "Client ID", 
    "Producto_ID": "Product ID", 
    "Venta_uni_hoy": "Sales Unit this week",
    "Venta_hoy": "Sales this week",
    "Dev_uni_proxima": "Returns Unit next week",
    "Dev_proxima": "Returns next week",
    "Demanda_uni_equil": "Demand"
}

In [12]:
train_data = train_data.rename(columns=columns)
test_data = test_data.rename(columns=columns)
product_data = product_data.rename(columns = columns)
client_data = client_data.rename(columns=columns)
town_state_data = town_state_data.rename(columns=columns)

## Data Exploration

### Town Data

In [13]:
town_state_data.head()

,Sales Depot ID,Town,State
0,1110,2008 AG. LAGO FILT,"MÉXICO, D.F."
1,1111,2002 AG. AZCAPOTZALCO,"MÉXICO, D.F."
2,1112,2004 AG. CUAUTITLAN,ESTADO DE MÉXICO
3,1113,2008 AG. LAGO FILT,"MÉXICO, D.F."
4,1114,2029 AG.IZTAPALAPA 2,"MÉXICO, D.F."


In [14]:
town_state_data["Sales Depot ID"].info()


<class 'pandas.core.series.Series'>
RangeIndex: 790 entries, 0 to 789
Series name: Sales Depot ID
Non-Null Count  Dtype
--------------  -----
790 non-null    int16
dtypes: int16(1)
memory usage: 1.7 KB


In [15]:
print("Our town state data has", town_state_data.shape[0], "observations and", town_state_data.shape[1], "variables.")

Our town state data has 790 observations and 3 variables.


In [16]:
town_state_data["town_id"] = town_state_data["Town"].str.split().str.get(0)
print("There are", town_state_data["town_id"].nunique(), "unique towns.")

There are 257 unique towns.


In [17]:
town_state_data["State"] = town_state_data["State"].apply(str.upper)
states = sorted(town_state_data["State"].unique())
print("There are", len(states), "states in our dataset.")
print("These states are", states)

There are 33 states in our dataset.
These states are ['AGUASCALIENTES', 'BAJA CALIFORNIA NORTE', 'BAJA CALIFORNIA SUR', 'CAMPECHE', 'CHIAPAS', 'CHIHUAHUA', 'COAHUILA', 'COLIMA', 'DURANGO', 'ESTADO DE MÉXICO', 'GUANAJUATO', 'GUERRERO', 'HIDALGO', 'JALISCO', 'MICHOACÁN', 'MORELOS', 'MÉXICO, D.F.', 'NAYARIT', 'NUEVO LEÓN', 'OAXACA', 'PUEBLA', 'QUERETARO', 'QUERETARO DE ARTEAGA', 'QUINTANA ROO', 'SAN LUIS POTOSÍ', 'SINALOA', 'SONORA', 'TABASCO', 'TAMAULIPAS', 'TLAXCALA', 'VERACRUZ', 'YUCATÁN', 'ZACATECAS']


We make note that Mexico itself has 32 states (including the capital, `MEXICO D.F.`) instead of 33 states. We notice that `QUERETARO DE ARTEAGA` was the old name of `QUERTARO`. 

In [18]:
town_state_data["State"] = town_state_data["State"].str.replace("QUERETARO DE ARTEAGA", "QUERTARO")

In [19]:
from sklearn.preprocessing import LabelEncoder

In [20]:
le = LabelEncoder()
town_state_data["state_id"] = le.fit_transform(town_state_data["State"])

In [21]:
town_state_data.head()

,Sales Depot ID,Town,State,town_id,state_id
0,1110,2008 AG. LAGO FILT,"MÉXICO, D.F.",2008,16
1,1111,2002 AG. AZCAPOTZALCO,"MÉXICO, D.F.",2002,16
2,1112,2004 AG. CUAUTITLAN,ESTADO DE MÉXICO,2004,9
3,1113,2008 AG. LAGO FILT,"MÉXICO, D.F.",2008,16
4,1114,2029 AG.IZTAPALAPA 2,"MÉXICO, D.F.",2029,16


In [22]:
town_state_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 790 entries, 0 to 789
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Sales Depot ID  790 non-null    int16 
 1   Town            790 non-null    object
 2   State           790 non-null    object
 3   town_id         790 non-null    object
 4   state_id        790 non-null    int32 
dtypes: int16(1), int32(1), object(3)
memory usage: 23.3+ KB


In [23]:
train_data["Client ID"].nunique()

880604

In [24]:
client_data["Client ID"].nunique()

930500

### Client Data

In [25]:
client_data.head(6)

,Client ID,NombreCliente
0,0,SIN NOMBRE
1,1,OXXO XINANTECATL
2,2,SIN NOMBRE
3,3,EL MORENO
4,4,SDN SER DE ALIM CUERPO SA CIA DE INT
5,4,SDN SER DE ALIM CUERPO SA CIA DE INT


We can see that our dataset have duplicate rows. Let us remove those rows.

In [26]:
client_data = client_data.drop_duplicates(keep="first", subset=["Client ID"]).reset_index()
del client_data["index"]

In [27]:
print("Our client dataset has", client_data.shape[0], "observations", "and", client_data.shape[1], "variables.")

Our client dataset has 930500 observations and 2 variables.


In [28]:
client_data.head(7)

,Client ID,NombreCliente
0,0,SIN NOMBRE
1,1,OXXO XINANTECATL
2,2,SIN NOMBRE
3,3,EL MORENO
4,4,SDN SER DE ALIM CUERPO SA CIA DE INT
5,5,LA VAQUITA
6,6,LUPITA


In [29]:
client_data["client_name_id"] = client_data.apply(
    lambda x: x["NombreCliente"] + "_" + str(x["Client ID"]) 
    if x["NombreCliente"] == "NO IDENTIFICADO" else x["NombreCliente"], 
    axis=1
)

In [30]:
le = LabelEncoder()
client_data["client_name_id"] = le.fit_transform(client_data["client_name_id"])
print("There are", client_data["client_name_id"].nunique(), "identifying clients.")

There are 588678 identifying clients.


In [31]:
client_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 930500 entries, 0 to 930499
Data columns (total 3 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Client ID       930500 non-null  int32 
 1   NombreCliente   930500 non-null  object
 2   client_name_id  930500 non-null  int32 
dtypes: int32(2), object(1)
memory usage: 14.2+ MB


### Product Data

In [32]:
product_data.head()

,Product ID,NombreProducto
0,0,NO IDENTIFICADO 0
1,9,Capuccino Moka 750g NES 9
2,41,Bimbollos Ext sAjonjoli 6p 480g BIM 41
3,53,Burritos Sincro 170g CU LON 53
4,72,Div Tira Mini Doradita 4p 45g TR 72


In [33]:
print("Our product data has", product_data.shape[0], "observations and", product_data.shape[1], "variables.")

Our product data has 2592 observations and 2 variables.


In [34]:
product_data["NombreProducto"] = product_data["NombreProducto"].str.replace(r"\s+\d+$", "", regex=True)
product_data["NombreProducto"] = product_data.apply(
    lambda x: x["NombreProducto"] + " " + "-1p" + " " + "-1g" + " " + "na"
    if x["NombreProducto"] == "NO IDENTIFICADO"
    else x["NombreProducto"], axis=1
)
product_data["units"] = product_data["NombreProducto"].str.extract(r" (-?\d+p) ")
product_data["size"] = product_data["NombreProducto"].str.extract(r" (-?\d+(?:g|kg|Kg|ml)) ")
product_data["brand"] = product_data["NombreProducto"].str.extract(r" ([^(?:g|l|p)]*$)")
product_data["product_name"] = product_data["NombreProducto"].str.extract(r"^(.*?)\s+-?\d+(?:g|ml|kg|Kg|p|P)\s+")

In [35]:
product_data["size_metric"] = product_data["size"].str.extract(r"(g|kg|Kg|ml)$").fillna("g")
product_data["size"] = product_data["size"].str.extract(r"(-?\d+)").fillna("-1").astype("int32")
product_data["weight"] = product_data.apply(
    lambda x: int(x["size"]) if x["size_metric"] == "g" or x["size_metric"] == "ml"
    else int(x["size"]) * 1000, axis=1
).astype("int32")
del product_data["size"]
del product_data["size_metric"]
product_data["weight"] = product_data["weight"].replace(-1, np.NaN)
product_data["units"] = product_data["units"].str.extract(r"(-?\d+)").fillna("-1").astype("int16")
product_data["units"] = product_data["units"].replace(-1, np.NaN)
product_data["brand"] = product_data["brand"].replace(r"^na$", np.NaN, regex=True)
del product_data["NombreProducto"]
product_data = product_data[["Product ID", "product_name", "brand", "weight", "units"]]

In [36]:
product_data.tail()

,Product ID,product_name,brand,weight,units
2587,49992,Tostado Integral,MTA WON,180.0,NaN
2588,49993,Tostado Integral,TAB WON,180.0,NaN
2589,49994,Tostado Int 0pct Grasa Azuc,WON,200.0,NaN
2590,49996,Tostado Int 0pct Grasa Azuc,MTA WON,200.0,NaN
2591,49997,Tostado Int 0pct Grasa Azuc,TAB WON,200.0,NaN


In [44]:
product_data[["weight", "units"]].describe()

,weight,units
count,2523.000000,1117.000000
mean,416.897344,10.567592
std,1770.309014,11.133603
min,4.000000,1.000000
25%,80.000000,4.000000
50%,180.000000,8.000000
75%,410.000000,12.000000
max,42000.000000,200.000000


In [37]:
product_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2592 entries, 0 to 2591
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Product ID    2592 non-null   int32  
 1   product_name  2549 non-null   object 
 2   brand         2590 non-null   object 
 3   weight        2523 non-null   float64
 4   units         1117 non-null   float64
dtypes: float64(2), int32(1), object(2)
memory usage: 91.2+ KB


### Train and Test data

In [38]:
train_data.head()

,wk_no,Sales Depot ID,Sales Channel ID,Route ID,Client ID,Product ID,Sales Unit this week,Sales this week,Returns Unit next week,Returns next week,Demand
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,3
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,4
2,3,1110,7,3301,15766,1238,4,39.320000,0,0.0,4
3,3,1110,7,3301,15766,1240,4,33.520000,0,0.0,4
4,3,1110,7,3301,15766,1242,3,22.920000,0,0.0,3


In [39]:
print("Our train data has", train_data.shape[0], "observations and", train_data.shape[1], "observations.")

Our train data has 74180464 observations and 11 observations.


In [40]:
train_data[["Sales Unit this week", "Sales this week", "Returns Unit next week", "Returns next week", "Demand"]].describe()

,Sales Unit this week,Sales this week,Returns Unit next week,Returns next week,Demand
count,7.418046e+07,7.418046e+07,7.418046e+07,7.418046e+07,7.418046e+07
mean,7.310163e+00,6.854449e+01,1.302577e-01,1.243248e+00,7.224564e+00
std,2.196734e+01,3.389795e+02,2.932320e+01,3.921552e+01,2.177119e+01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.000000e+00,1.676000e+01,0.000000e+00,0.000000e+00,2.000000e+00
50%,3.000000e+00,3.000000e+01,0.000000e+00,0.000000e+00,3.000000e+00
75%,7.000000e+00,5.610000e+01,0.000000e+00,0.000000e+00,6.000000e+00
max,7.200000e+03,6.473600e+05,2.500000e+05,1.307600e+05,5.000000e+03


In [41]:
test_data.head()

,id,wk_no,Sales Depot ID,Sales Channel ID,Route ID,Client ID,Product ID
0,0,11,4037,1,2209,4639078,35305
1,1,11,2237,1,1226,4705135,1238
2,2,10,2045,1,2831,4549769,32940
3,3,11,1227,1,4448,4717855,43066
4,4,11,1219,1,1130,966351,1277


In [42]:
print("Our test data has", test_data.shape[0], "observations and", test_data.shape[1], "observations.")

Our test data has 6999251 observations and 7 observations.


In [43]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6999251 entries, 0 to 6999250
Data columns (total 7 columns):
 #   Column            Dtype
---  ------            -----
 0   id                int64
 1   wk_no             int8 
 2   Sales Depot ID    int16
 3   Sales Channel ID  int8 
 4   Route ID          int16
 5   Client ID         int32
 6   Product ID        int32
dtypes: int16(2), int32(2), int64(1), int8(2)
memory usage: 146.9 MB


### Putting our data into a csv file

In [45]:
import os

In [46]:
parent_dir = os.getcwd()
path = os.path.join(parent_dir, "Stage_1-Data Exploration")
if os.path.exists(path) != True: 
   os.mkdir(path)

os.chdir(path)
town_state_data.to_csv("town_state.csv")
client_data.to_csv("client.csv")
product_data.to_csv("product.csv")
train_data.to_csv("train.csv")
test_data.to_csv("test.csv")